# Clustering of websites by text

Thanks to the scrapping notebook I could gather the text of the given websites and the links in the homepage and write it into a text file. 

There is one text file per site. 

While most of the 50 sites were scrapped succesfully, there a few I couldn't get. 

Out of 50 sites:

2 couldn't be requested properly.
5 gave a 403 or 503 http error code. 
1 was requested, but the script recovered the text of the loading page. 

I will start the clustering with the data I could gather, and go back to the missing websites after the clustering process is well stablished.

My first step is to load the text data and load it into a dataframe, in order to start exploring it. 

In [1]:
import os
import re

import pandas as pd
import numpy as np

from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score, calinski_harabasz_score, davies_bouldin_score

import matplotlib.pyplot as plt
from matplotlib import cm


%matplotlib inline

In [2]:
# Set the contents directory
CONTENTS_DIR = './site_contents/'

In [3]:
# Get all file names from the directory.

file_names = [file for file in os.listdir(CONTENTS_DIR)]

In [4]:
# Read the text of each file
file_contents = []

for name in file_names:
    
    with open(CONTENTS_DIR + name, 'r') as content:
        site_text = content.read()
    
    file_contents.append(site_text)

In [5]:
# Build a dataframe to store the site names and the extracter text
websites_df = pd.DataFrame({'site': map(lambda name: name.replace('.txt','' ), file_names),
                            'raw_text': file_contents})

In [7]:
# Some info on the dataframe.

websites_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 43 entries, 0 to 42
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   site      43 non-null     object
 1   raw_text  43 non-null     object
dtypes: object(2)
memory usage: 816.0+ bytes


In [ ]:
websites_df.head(5)

In [ ]:
# Which languages are have stopwords list in nltk?
print(f'There are a total of {len(stopwords.fileids())} languages: \n')
print(', '.join(stopwords.fileids()))

In [ ]:
# Build list of all stopwords:

stop_words = []
for lan in stopwords.fileids():
    stop_words.extend(stopwords.words(lan))

print(f'Gathered a total of {len(stop_words)} stopwords.')

In [ ]:
# Get a list of curated html tags that may have remained.
# The list contains all html tags, and the ones that shouldn't be included
# as stopwords are preceded by the simbol '#'.

with open('filter_tags', 'r') as tags:
    stop_words.extend([tag.strip() for tag in tags.readlines() if (not tag.startswith('#') and len(tag.strip())>=3)])

In [ ]:
# Let's add to the stopwords a list of file types and html tags, in case something got through

file_types = ['pdf', 'jpg', 'jpeg', 'png', 'gif', 'exe', 'js', 'zip', 'tar', 'gz', '7z', 'rar']

stop_words.extend(file_types)

In [ ]:
len(list(set(stop_words)))

In [10]:
websites_df['raw_text'] = websites_df['raw_text'].apply(lambda raw: re.sub(r'\b\d+\b', '',raw))

In [11]:
# Add column with split text, and one with the length of the split text.
websites_df['wordcount']=websites_df['raw_text'].apply(lambda mytext: len(mytext.split()))


In [12]:
websites_df.describe()

,wordcount
count,43.000000
mean,15707.116279
std,23752.146283
min,0.000000
25%,5683.000000
50%,10214.000000
75%,16095.500000
max,146517.000000


In [ ]:
websites_df.info()

In [ ]:
# Some general information on the wordcounts.
websites_df.describe()

In [ ]:
websites_df.sort_values('wordcount').head(10)

In [ ]:
websites_df.drop(websites_df[websites_df['wordcount']< 100].index, inplace=True)

In [ ]:
websites_df.info()

In [ ]:
# Lets see how the wordcount is distributed.

websites_df['wordcount'].apply(np.log10).hist(bins = 20, figsize=(7,7))
plt.title('Word count distribution')
plt.xlabel('Log10(wordcount)')
plt.show()

In [ ]:
# Viewing the 10 websites with the lowest wordcounts.

websites_df.sort_values('wordcount').head(10)

We can also see the sites with higher wordcounts.

In [ ]:
websites_df.sort_values('wordcount').tail(10)

From the table above we see that there are a few websites with very few words, and that they correspond mostly to failures in getting the content of the page. We should get rid of these sites, and a good way is to drop those with a low wordcount.

In [ ]:
# Set minimun amount of word required to remain in the data set. 
MIN_WORDS = 100

# Drop sites with less words than MIN_WORDS.
websites_df.drop(websites_df[ websites_df['wordcount'] < MIN_WORDS ].index, inplace=True)

In [ ]:
# Let's see what we have left.

websites_df.shape

In [ ]:
# Viewing the 10 websites with the lowest wordcounts after drop.

websites_df.sort_values('wordcount').head(10)

In [ ]:
# Lets see how the wordcount is distributed.

websites_df['wordcount'].apply(np.log10).hist(bins = 20, figsize=(7,7))
plt.title('Word count distribution')
plt.xlabel('Log10(wordcount)')
plt.show()

We still have very different lenghts, but even a few hundred words could help us classify a site. 

Before tokenizing and vectorizing, it would be good some things that are likely to appear many times but not give us a lot of information. Since we are dealing with different languages, some language-specific pre-processing techiniques like removing stopwords or lematization would be harder to implement. 

Some things that can appear in the websites and won't give much info. 
* Pure numbers.
* Pure symbols like +, - ?, &, etc. (Beware of not removing non-latin alphabet's characters. )
* email adresses and URLs. 

The idea behind removing these elements is the intuition that sites will be clustered by language first, and then perhaps by topic within each language. The information relative to these elemenets most likely doesn't depend on the above elements. 

If I assume I know at least what some of the languages are, I can remove some of the stopwords. But this assumes knowledge that I'm not sure I have available. I will leave this aside for the time being, but it could be dealt with on a future version of the process.  

## Vectorizing text with Tf-Idf

I will use Tf-Idf to vectorize my texts. This will build a vocabulary, and assing a vector to each document. The i-th coordinate of the vector is the product between the Term Frequency and the Inverse Document Frecuency of the i-th word in the vocabulary.

Furthermore, this vectorizer will normalize all vectors to length 1, which will reduce the efect of comparing texts of very different lengths. 

In [ ]:
# Creating a vectorizer object. We can pass parameter to control certain aspects
# of the vectorization process. For now we use the default values. 

# Changed token patterns to keep words with 3 or more letters only.
vectorizer = TfidfVectorizer(min_df =2,
                             max_df = 0.5,
                             stop_words=stop_words,
                             token_pattern = '(?u)\\b\\w\\w\\w+\\b', ngram_range=(1,2) )

In [ ]:
""" The default regexp select tokens of 2 or more alphanumeric characters 
(punctuation is completely ignored and always treated as a token separator)."""

In [ ]:
# Building vocabulary and vectors

doc_vectors = vectorizer.fit_transform(websites_df['raw_text'])

print(f'We obtained a vocabulary of {len(vectorizer.vocabulary_)} different words and bigrams.')

In [ ]:
print(f'We identified {len(vectorizer.stop_words_)} words or bigrams that appear in only one document, or in more than half' )  

In [ ]:
vectorizer.stop_words_

The vectors obtained in this way are stored in a sparse matrix (i.e. it's entries are mostly 0).

Each row corresponds to a document, and each column to a word in the vocabulary.

In [ ]:
print(f'The dimensions of the document matrix are {doc_vectors.shape} .')

###  Visualizing the document vectors

With the documents matrix built, we can explore it superficially in order to get an idea of the structures of our documents. 

We show two visualizations: The non-zero elements of the matrix, and the cosine similarity matrix. 

In [ ]:
# Visualizing the non-zero elements of the matrix. 

plt.figure(figsize=(10, 5))
plt.spy(doc_vectors, markersize=1, aspect = 'auto')
plt.title('Document matrix non-zero elements')
plt.xlabel('Word in vocabulary')
plt.ylabel('Document number')
plt.show()

From this plot we can see there are two documents (numbers 13 and 32) containtn a lot of words (they look like horizontal lines) and one (number 9) with very few. The rest of them seem mostly balanced, but in any case shouldn't be a major factor. 

From this bird's eye view of our data we cannont distinguish any clear cluster structure.
This may seem suprising, as one might expect that at least sites would naturally be clustered by language. However, words in the vocabulary are not grouped by language, hiding this effect. 

In [ ]:
# Computing cosine similarities. Since verctors are normalized to 1, 
# it suffices to multiply the matrix by its transpose. 

cosine_sims = doc_vectors * doc_vectors.transpose()

In [ ]:
# Heatmap of the cosine similarities. 
# There are no correletions higher than 0.6 (other than the diagonal), 
# So we choose this value for a cutoff of the scale.

my_cmap = cm.get_cmap('viridis')
plt.figure(figsize=(10,10))
plt.imshow(cosine_sims.toarray(), cmap=my_cmap, vmax=.8)
plt.colorbar(shrink = 0.8)

plt.title('Cosine Similarities between documents')

plt.show()

##  Dimensional reduction with PCA

Reducing dimensionality can help the KMeans algorithm work better, as it relies on euclidean distances. 

The price to pay for this is that we may loose some interpretability, as the components of our dimensions will be mixed. 

Let's try ir out!

In [ ]:
# Let's keep the 2 principal components, and visualize the data to see if we see something.
# I think this is a long shot.

# Set random state for reproducibility.
decomposer_2d = TruncatedSVD(random_state=42)

In [ ]:
docs_2d = decomposer_2d.fit_transform(norm_docs)
decomposer_2d.explained_variance_ratio_

We can get the 10 most important words for our principal components.

In [ ]:
# Invert the vocabulary dictionary
inv_vocab = {v:k for k,v in vectorizer.vocabulary_.items()}

# Sort component arguments by descending order of the components values.

word_ids_1 = np.argsort(decomposer_2d.components_[0,:])[::-1]
word_ids_2 = np.argsort(decomposer_2d.components_[1,:])[::-1]

In [ ]:
for i, ids in enumerate([word_ids_1, word_ids_2]):
    words = [inv_vocab[w_id] for w_id in ids[:10]] 
    
    print(f'Words of dimension {i} :\n')
    print(' | '.join(words))
    print('\n'+'='*20+'\n')

In [ ]:
plt.figure(figsize=(10,10))
plt.plot(decomposer_2d.components_[0,:])
plt.plot(decomposer_2d.components_[1,:])
plt.show()

In [ ]:
plt.figure(figsize = (10,10))
plt.scatter(docs_2d[:,0], docs_2d[:,1])
plt.show()

###  Keeping 10 principal components

In [ ]:
decomposer_10 = TruncatedSVD(n_components = 10)

decomposer_10.fit(doc_vectors)
docs_10d = decomposer_10.transform(doc_vectors)

decomposer_10.singular_values_

In [ ]:
decomposer_10.explained_variance_

###  Keeping the 100 first principa components

In [ ]:
decomposer_100 = TruncatedSVD(n_components = 100)

In [ ]:
decomposer_100.fit(doc_vectors)
docs_100d = decomposer_100.transform(doc_vectors)

In [ ]:
decomposer_100.singular_values_

In [ ]:
len(decomposer_100.components_)

In [ ]:
docs_100d.shape

In [ ]:
plt.plot(decomposer_100.explained_variance_)
plt.title('Explained variance of each component')
plt.show()

Let's cluster using these reduced vectors.

##  Clustering using KMeans

In [ ]:
# Create a model

KMeans_model = KMeans(n_clusters = 5, n_init=500, max_iter= 100)

In [ ]:
# Find the clusters
KMeans_model.fit(norm_docs)

# Write the labels into the dataframe.
websites_df['cluster_label'] =  KMeans_model.labels_

# How many elements in each cluster?
websites_df.groupby('cluster_label')['site'].count()

In [ ]:
# Let's see cosine simlarities ordered by cluster

sorted_vecs = vectorizer.transform(websites_df.sort_values('cluster_label')['raw_text'])


In [ ]:
ordered_indices = websites_df.sort_values('cluster_label')['cluster_label']
n_sites = websites_df.shape[0]
clabel = np.zeros((n_sites, n_sites), dtype = int) -1

In [ ]:
for i in range(n_sites):
    for j in range(n_sites):
        if ordered_indices.iloc[i] == ordered_indices.iloc[j]:
            clabel[i,j] = ordered_indices.iloc[i]
            
clabel

In [ ]:
# Sorted cosine sims
sorted_sims = sorted_vecs * sorted_vecs.transpose()

my_cmap = cm.get_cmap('viridis')
plt.figure(figsize=(15,15))
plt.imshow(sorted_sims.toarray(), cmap=my_cmap, vmax=0.9)
plt.colorbar(shrink = 0.8)

for i in range(n_sites):
    for j in range(n_sites):
        if clabel[i, j] >=0:
            text = plt.text(j, i, clabel[i, j],
                           ha="center", va="center", color="w")

plt.title('Cosine Similarities between documents')

plt.show()


In [ ]:
KMeans_model.labels_

In [ ]:
from mpl_toolkits.mplot3d import Axes3D
import seaborn as sns

In [ ]:
%matplotlib notebook
fig = plt.figure(figsize=(10,10))
ax = fig.add_subplot(111, projection='3d')

x = docs_10d[:,0]
y = docs_10d[:,1]
z = docs_10d[:,2]



ax.scatter(x, y, z, c=KMeans_model.labels_, marker='o')

ax.set_xlabel('X Label')
ax.set_ylabel('Y Label')
ax.set_zlabel('Z Label')

plt.show()

In [ ]:
websites_df[websites_df['cluster_label']==4]

In [ ]:
inv_vocab = {v:k for k,v in vectorizer.vocabulary_.items()}

In [ ]:
cluster_dict = {}
for i in range(KMeans_model.cluster_centers_.shape[0]):
    vec = KMeans_model.cluster_centers_[i]
    argsorted = np.argsort(vec)[::-1]
    
    cluster_dict[f'{i}'] = [inv_vocab[i] for i in argsorted[:15]]

In [ ]:
for k, v in cluster_dict.items():
    print(f'Cluster {k} most important words \n')
    print(' | '.join(v))
    print('\n\n'+'='*20 )

In [ ]:
from sklearn.metrics import silhouette_score, calinski_harabasz_score, davies_bouldin_score

In [ ]:
# Silhouette should be close to 1
# C-H should be as big as possible
# D-B should be close to 0
# Inertia should be minimized. 

scores_dict = {'sil': [],
               'ch': [],
               'db': [],
               'inertia': []
              }

#doc_vec_array = doc_vectors.toarray()

mydocs = doc_vectors.toarray()

cnumbers = range(2,int(doc_vectors.shape[0]/3+1))
for c_number in cnumbers:
    
    n_model = KMeans(n_clusters=c_number, n_init= 100, max_iter=100)
    n_model.fit(mydocs)
    labels = n_model.labels_
    # Compute scores

    model_scores = {'sil': silhouette_score(mydocs, labels),
                    'ch': calinski_harabasz_score(mydocs, labels),
                    'db': davies_bouldin_score(mydocs, labels),
                    'inertia': n_model.inertia_}

    for index_name, score in model_scores.items():
        scores_dict[index_name].append(score)

In [ ]:
%matplotlib inline
plt.plot(cnumbers, scores_dict['inertia'])


In [ ]:
plt.plot(cnumbers, scores_dict['sil'])

In [ ]:
geo_mean = [np.sqrt(sil * ch) for sil, ch in zip(scores_dict['sil'], scores_dict['ch'])]

In [ ]:
plt.plot(geo_mean)

In [ ]:
plt.plot(cnumbers, scores_dict['ch'])

In [ ]:
plt.plot(cnumbers, scores_dict['db'])

In [ ]:
file_names=['www.pelopincho.com.txt']
file_contents = []

for name in file_names:
    
    with open(CONTENTS_DIR + name, 'r') as content:
        site_text = content.read()
    
    file_contents.append(site_text)

In [ ]:
pelopincho_df = pd.DataFrame({'site': map(lambda name: name.replace('.txt','' ), file_names),
                            'raw_text': file_contents})

In [ ]:
pelopincho_df

In [ ]:
pelopincho_df['raw_text'] = websites_df['raw_text'].apply(lambda raw: re.sub(r'\b\d+\b', '',raw))

In [ ]:
pelopincho_vec =vectorizer.transform(pelopincho_df['raw_text'])

In [ ]:
label = KMeans_model.predict(pelopincho_vec)

In [ ]:
label